## Data augmentation과 Dropout 의 DNN 적용

일반적으로, 과적합을 방지하기 위해서 Dropout 층을 이용한다. Drop out 의 경우, 일부에만 역전파를 적용하여, 학습에 절반정도의 신경망만 사용하는 방법이다. 
본문에서는 Pytorch 에서 Dropout 과 Data Augmentation 의 적용 코드를 다룬다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.utils import data
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Torch vision 의 transforms 는 입력을 변환시키는 도구로, Tensor 변환, Resize(), Normalize(), 등의 편리한 기능을 많이 가지고 있다.
#사용법은 다음과 같다.
#Data Augmentation을 위하여,transforms.Compose() 함수에 transforms.RandomHorizontalFlip 을 이용하여 무작위 수평 뒤집기를 수행하였다.
#Test Transform 은 변환하지 않는다.

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])

In [3]:
EPOCHS=50
BATCH_SIZE=64

In [4]:
trainset = datasets.FashionMNIST(root = './data/', train = True, download = True, transform = train_transform)
testset = datasets.FashionMNIST(root = './data/', train = False, download = True, transform = test_transform)

In [5]:
train_loader = data.DataLoader(dataset=trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
#GPU 사용 코드 CUDA 
USE_CUDA = torch.cuda.is_available() 
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [7]:
class Net(nn.Module):
    #__init__ 변수에 Dropout 이 추가 되었다.
    def __init__(self, dropout_p = 0.2):
        super(Net, self).__init__()
        # 층의 입력과 출력을 변수로 둔다.
        self.fc1 = nn.Linear(784,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,10)
        
        #Dropout 확률을 변수로 둔다.
        self.dropout_p = dropout_p        
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x)) #가중치가 없는 연산의 경우 torch.nn.functional 에 있는 함수를 사용하기도 한다.
        # 드롭 아웃층 추가
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training, p=self.dropout_p)
        x = self.fc3(x)
        return x

In [8]:
model = Net(dropout_p=0.2).to(DEVICE) #모델을 cuda 메모리로 보내 처리하기 위해선 to() 함수를 써야한다.
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
def train(model, train_loader, optimizer):
    model.train() #학습/ 평가 모드에 따라 동작이 다른 모듈이 있기에 학습 모드로 전환 (항상 사용해 버릇 하는 것이 좋음)
        
    for batch_idx, (data, target) in enumerate(train_loader):
        #학습 데이터를 DEVICE 메모리로 전달한다.
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
        
    #평가과정에서는 기울기를 계산할 필요가 없기에 torch.no_grad() 를 명시해주어야함
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
                
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
                
            # 가장 큰 값을 가진 클래스가 모델의 예측임
            # 예측과 정답을 비교하여 일치할경우 coreect에 1 을 더함
            pred = output.max(1, keepdim=True)[1] #output.max() 함수는 가장 큰값(확률)과 그 값이 있는 인덱스를 추출한다.
            correct += pred.eq(target.view_as(pred)).sum().item() #eq() 함수는 일치하면 1을 아니면 0 을 출력함
        
    #전체 데이터셋에 대한 오차와 맞힌 개수의 합
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.* correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6438, Accuracy: 77.44%
[2] Test Loss: 0.5354, Accuracy: 80.82%
[3] Test Loss: 0.4945, Accuracy: 81.94%
[4] Test Loss: 0.4614, Accuracy: 83.31%
[5] Test Loss: 0.4494, Accuracy: 84.03%
[6] Test Loss: 0.4322, Accuracy: 84.39%
[7] Test Loss: 0.4134, Accuracy: 84.96%
[8] Test Loss: 0.4053, Accuracy: 85.53%
[9] Test Loss: 0.4002, Accuracy: 85.63%
[10] Test Loss: 0.3907, Accuracy: 85.76%
[11] Test Loss: 0.3848, Accuracy: 86.04%
[12] Test Loss: 0.3788, Accuracy: 86.33%
[13] Test Loss: 0.3741, Accuracy: 86.36%
[14] Test Loss: 0.3674, Accuracy: 86.74%
[15] Test Loss: 0.3676, Accuracy: 86.89%
[16] Test Loss: 0.3608, Accuracy: 87.06%
[17] Test Loss: 0.3619, Accuracy: 86.84%
[18] Test Loss: 0.3523, Accuracy: 87.35%
[19] Test Loss: 0.3565, Accuracy: 87.39%
[20] Test Loss: 0.3487, Accuracy: 87.58%
[21] Test Loss: 0.3512, Accuracy: 87.34%
[22] Test Loss: 0.3424, Accuracy: 87.90%
[23] Test Loss: 0.3540, Accuracy: 87.17%
[24] Test Loss: 0.3512, Accuracy: 87.37%
[25] Test Loss: 0.3368, A